In [ ]:
!pip install CausalInference
!pip uninstall numpy
!pip install numpy==1.18.5
!pip install causalml
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from causalinference import CausalModel

from IPython.display import Image
from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot
from causalml.inference.meta import LRSRegressor

Uninstalling numpy-1.18.5:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.6
    /usr/local/lib/python3.6/dist-packages/numpy-1.18.5.dist-info/*
    /usr/local/lib/python3.6/dist-packages/numpy.libs/libgfortran-ed201abd.so.3.0.0
    /usr/local/lib/python3.6/dist-packages/numpy.libs/libopenblasp-r0-34a18dc3.3.7.so
    /usr/local/lib/python3.6/dist-packages/numpy/*
Proceed (y/n)? y
  Successfully uninstalled numpy-1.18.5
  Using cached https://files.pythonhosted.org/packages/b3/a9/b1bc4c935ed063766bce7d3e8c7b20bd52e515ff1c732b02caacf7918e5a/numpy-1.18.5-cp36-cp36m-manylinux1_x86_64.whl
ERROR: tensorflow 2.4.0 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
DATA_PATH = '/content/gdrive/MyDrive/Colab Notebooks/AB-data'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 1. Оценка результатов AB теста
В данном задании вам требуется оценить результаты AB теста. Даны результаты на контрольной и тестовой группе после проведения AB теста. Помимо показателей целевой переменной, вам даны также несколько фичей, которые являются характеристиками объектов. Даны целевые показатели (y) и принадлежность к целевой группе (W).

Задача:
1. Проверьте подходящую для данных гипотезу о равенстве средних в выборках
2. Каков treatment effect нашего воздействия? (ATE)
3. Примените известные вам методы оценки ATE с учетом особенностей данных.
4. Проанализируйте, сравните результаты применения различных методов.
5. В явном виде выпишите выводы.
*Примечание: предположения о рандомизированном эксперименте нуждаются в проверке - независимость treatment от объектов не гарантирована. Все предположения выписывайте явно.*

Для данной задачи используется dataset: dataset_part1_students.csv

In [ ]:
data = pd.read_csv('{}/dataset_part1_students.csv'.format(DATA_PATH), sep='\t')
data.head()

,y,W,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10
0,-1.459404,0.0,0.986277,0.873392,0.509746,0.271836,0.336919,0.216954,0.276477,0.343316,0.862159,0.156700
1,-1.694174,0.0,0.140887,0.757080,0.736325,0.355663,0.341093,0.666803,0.217101,0.561427,0.124179,0.319736
2,5.378374,0.0,0.953214,0.137357,0.569413,0.975665,0.503367,0.667664,0.034191,0.456119,0.155851,0.476049
3,2.188189,1.0,0.169702,0.896258,0.373394,0.379693,0.858317,0.646061,0.583462,0.668350,0.177793,0.849248
4,2.862696,1.0,0.442373,0.831468,0.763921,0.919691,0.070573,0.156165,0.636894,0.555696,0.191929,0.425656


In [ ]:
def t_test(target):
  # Проверю гипотезу о равенстве средних в выборках
  control_group = data[data['W'] == 0][target].values # контрольная группа
  test_group = data[data['W'] == 1][target].values # тестовая группа
  alpha = 0.05

  T = (abs(control_group.mean() - test_group.mean())) / np.sqrt(control_group.std(ddof=1)**2/control_group.shape[0] + test_group.std(ddof=1)**2/test_group.shape[0])
  st = stats.t.ppf(1 - alpha, df=control_group.shape[0] + test_group.shape[0] - 2)
  p_value = (1 - stats.t.cdf(T, df=control_group.shape[0] + test_group.shape[0] - 2)) * 2

  print('t:', round(T, 4))
  print('st:', round(st, 4))
  #print('p value:', round(p_value, 4))
  if T < st:
    print('T < St => принимается нулевая гипотеза о равенстве средних в выборках')
  else:
    print('T >= St => нулевая гипотеза отвергается в пользу альтернативы')

t_test('y')

t: 9.3326
st: 1.6449
T >= St => нулевая гипотеза отвергается в пользу альтернативы


In [ ]:
def treatment_effect(target):
  # treatment effect
  control_group = data[data['W'] == 0][target].values # контрольная группа
  test_group = data[data['W'] == 1][target].values # тестовая группа
  ate = test_group.mean() - control_group.mean()
  print('treatment effect:', round(ate, 3))

treatment_effect('y')

treatment effect: 0.4


In [ ]:
def ate_regression(target, ind):
  # ate регрессия
  x = data.iloc[:, ind:]
  y = data[target]
  X = sm.add_constant(x)
  model = sm.OLS(y,X)
  results = model.fit()
  return results.summary()

ate_regression('y', 1)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     28.94
Date:                Wed, 23 Dec 2020   Prob (F-statistic):           3.99e-61
Time:                        11:38:48   Log-Likelihood:                -50425.
No. Observations:               20000   AIC:                         1.009e+05
Df Residuals:                   19988   BIC:                         1.010e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4198      0.119     11.981      0.000       1.188       1.652
W              0.5134      0.049     10.467      0.000       0.417       0.610
X_1           -0.2255      0.080     -2.832      0.005      -0.382      -0.069
X_2           -0.3571      0.079     -4.502      0.000      -0.513      -0.202
X_3           -0.0980      0.073     -1.333      0.183      -0.242       0.046
X_4            0.9345      0.074     12.637      0.000       0.790       1.079
X_5            0.4214      0.074      5.707      0.000       0.277       0.566
X_6            0.1857      0.074      2.519      0.012       0.041       0.330
X_7            0.0839      0.074      1.134      0.257      -0.061       0.229
X_8           -0.1041      0.074     -1.409      0.159      -0.249       0.041
X_9           -0.1173      0.074     -1.587      0.112      -0.262       0.028
X_10           0.0494      0.074      0.667      0.505      -0.096       0.194
==============================================================================
Omnibus:                        0.441   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.802   Jarque-Bera (JB):                0.460
Skew:                          -0.009   Prob(JB):                        0.795
Kurtosis:                       2.984   Cond. No.                         12.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Применение линейной регрессии для вычисления ate дало результат 0.5134

Данный результат примерно согласуется с найденным наивным значением ate 0.4 выявляя положительное влияние контрольной группы

In [ ]:
causal = CausalModel(
    Y=data['y'], D=data['W'], X=data.iloc[:, 2:]
)

causal.est_via_ols(adj=1)
print(causal.estimates)


Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      0.513      0.049     10.458      0.000      0.417      0.610



### 2. Оценка результатов AB теста с показателями до начала эксперимента
Необходимо как и ранее оценить результаты AB теста, а именно ATE. Вы располагаете результатами до и после проведения теста (y_0 и y_1), а также вам дан treatment assignment (W).

Необходимо:

1. Проверить все требуемые гипотезы
2. Оценить ATE подходящим для этого случая методом (примените несколько методов)
3. Сравнить и проанализировать результаты оценок
4. Явно выписать выводы.
5. Как и прежде, все предположения должны быть указаны. Визуализация результатов приветствуется.

dataset_part2_students.csv

In [ ]:
data = pd.read_csv('{}/dataset_part2_students.csv'.format(DATA_PATH), sep='\t')
data.head()

,y_1,y_0,W,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10
0,5.887156,2.785060,1.0,0.605978,0.733369,0.138947,0.312673,0.997243,0.128162,0.178993,0.752925,0.662161,0.784310
1,2.232601,5.533515,0.0,0.096894,0.058571,0.962396,0.616557,0.086630,0.561272,0.616525,0.963843,0.574304,0.371161
2,0.144419,0.095546,1.0,0.452145,0.201850,0.569305,0.195096,0.583704,0.476313,0.517814,0.823099,0.732225,0.069056
3,-2.446425,3.590472,1.0,0.672129,0.643485,0.828014,0.204469,0.617489,0.617701,0.301069,0.871741,0.589654,0.981770
4,2.163090,1.365405,0.0,0.442232,0.126318,0.508831,0.431786,0.915940,0.709016,0.890655,0.588886,0.636830,0.342209


In [ ]:
# t-тест
t_test('y_0')

# treatment effect
treatment_effect('y_0')

# ate regression
ate_regression('y_0', 2)

t: 2.7014
st: 1.645
T >= St => нулевая гипотеза отвергается в пользу альтернативы
treatment effect: -0.124


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    y_0   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     23.53
Date:                Wed, 23 Dec 2020   Prob (F-statistic):           1.80e-48
Time:                        11:39:04   Log-Likelihood:                -22357.
No. Observations:               10000   AIC:                         4.474e+04
Df Residuals:                    9988   BIC:                         4.482e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4724      0.127      3.734      0.000       0.224       0.720
W              0.0057      0.052      0.109      0.913      -0.096       0.108
X_1           -0.3480      0.085     -4.096      0.000      -0.515      -0.181
X_2           -0.2679      0.085     -3.166      0.002      -0.434      -0.102
X_3           -0.4370      0.078     -5.597      0.000      -0.590      -0.284
X_4            0.9364      0.078     11.965      0.000       0.783       1.090
X_5            0.5187      0.078      6.633      0.000       0.365       0.672
X_6           -0.0196      0.078     -0.250      0.803      -0.173       0.134
X_7            0.1285      0.078      1.641      0.101      -0.025       0.282
X_8           -0.0463      0.078     -0.592      0.554      -0.200       0.107
X_9            0.0206      0.079      0.261      0.794      -0.134       0.176
X_10           0.0633      0.079      0.803      0.422      -0.091       0.218
==============================================================================
Omnibus:                        1.502   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.472   Jarque-Bera (JB):                1.533
Skew:                          -0.026   Prob(JB):                        0.465
Kurtosis:                       2.970   Cond. No.                         13.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

В данном тесте можно увидеть расхождения в результатах. Наивный алгортим ate дал результат -0.124, демонстрируя негативный эффект. В то время как ate с помощью регрессии дал результат 0.0057, что говорит нам о минимальном положительном эффекте.

In [ ]:
# t-тест
t_test('y_1')

# treatment effect
treatment_effect('y_1')

# ate regression
ate_regression('y_1', 2)

t: 10.4305
st: 1.645
T >= St => нулевая гипотеза отвергается в пользу альтернативы
treatment effect: 0.431


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    y_1   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     39.48
Date:                Wed, 23 Dec 2020   Prob (F-statistic):           2.63e-84
Time:                        11:39:06   Log-Likelihood:                -21277.
No. Observations:               10000   AIC:                         4.258e+04
Df Residuals:                    9988   BIC:                         4.266e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1967      0.114     10.538      0.000       0.974       1.419
W              0.5482      0.047     11.724      0.000       0.457       0.640
X_1           -0.2645      0.076     -3.469      0.001      -0.414      -0.115
X_2           -0.2952      0.076     -3.886      0.000      -0.444      -0.146
X_3           -0.1394      0.070     -1.989      0.047      -0.277      -0.002
X_4            0.9812      0.070     13.967      0.000       0.844       1.119
X_5            0.6305      0.070      8.982      0.000       0.493       0.768
X_6            0.0787      0.070      1.117      0.264      -0.059       0.217
X_7            0.2004      0.070      2.850      0.004       0.063       0.338
X_8            0.1130      0.070      1.610      0.107      -0.025       0.251
X_9           -0.0421      0.071     -0.593      0.553      -0.181       0.097
X_10          -0.0226      0.071     -0.319      0.750      -0.161       0.116
==============================================================================
Omnibus:                        1.919   Durbin-Watson:                   1.987
Prob(Omnibus):                  0.383   Jarque-Bera (JB):                1.906
Skew:                           0.034   Prob(JB):                        0.385
Kurtosis:                       3.006   Cond. No.                         13.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Эффект от воздействия имеется, 0.431 для наивного алгоритма и 0.5482 для регрессии. Отсюда можно сделать вывод о правильно вычисленном ate на выборке после проведения теста

In [ ]:
# Проверю гипотезу о равенстве средних в выборках y0 и y1
control_group = data['y_0'].values # контрольная группа
test_group = data['y_1'].values # тестовая группа

alpha = 0.05

T = (abs(control_group.mean() - test_group.mean())) / np.sqrt(control_group.std(ddof=1)**2/control_group.shape[0] + test_group.std(ddof=1)**2/test_group.shape[0])
st = stats.t.ppf(1 - alpha, df=control_group.shape[0] + test_group.shape[0] - 2)
p_value = (1 - stats.t.cdf(T, df=control_group.shape[0] + test_group.shape[0] - 2)) * 2

print('t:', round(T, 4))
print('st:', round(st, 4))
#print('p value:', round(p_value, 4))
if T < st:
  print('T < St => принимается нулевая гипотеза о равенстве средних в выборках')
else:
  print('T >= St => нулевая гипотеза отвергается в пользу альтернативы')

t: 43.7587
st: 1.6449
T >= St => нулевая гипотеза отвергается в пользу альтернативы


In [ ]:
# treatment effect
ate = test_group.mean() - control_group.mean()
print('treatment effect (для y0):', round(ate, 3))

treatment effect (для y0): 1.353


In [ ]:
# ate регрессия
x = pd.DataFrame(np.concatenate((data.iloc[:, 2:], data.iloc[:, 2:])), columns=data.columns[2:])
y = np.concatenate((data['y_0'], data['y_1']))
X = sm.add_constant(x)
model = sm.OLS(y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     47.54
Date:                Wed, 23 Dec 2020   Prob (F-statistic):          1.08e-103
Time:                        11:39:09   Log-Likelihood:                -44680.
No. Observations:               20000   AIC:                         8.938e+04
Df Residuals:                   19988   BIC:                         8.948e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8346      0.089      9.348      0.000       0.660       1.010
W              0.2769      0.037      7.534      0.000       0.205       0.349
X_1           -0.3063      0.060     -5.108      0.000      -0.424      -0.189
X_2           -0.2816      0.060     -4.714      0.000      -0.399      -0.164
X_3           -0.2882      0.055     -5.231      0.000      -0.396      -0.180
X_4            0.9588      0.055     17.360      0.000       0.851       1.067
X_5            0.5746      0.055     10.412      0.000       0.466       0.683
X_6            0.0295      0.055      0.533      0.594      -0.079       0.138
X_7            0.1644      0.055      2.975      0.003       0.056       0.273
X_8            0.0334      0.055      0.605      0.545      -0.075       0.142
X_9           -0.0107      0.056     -0.192      0.848      -0.120       0.099
X_10           0.0203      0.056      0.366      0.714      -0.089       0.129
==============================================================================
Omnibus:                       29.439   Durbin-Watson:                   1.817
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               29.520
Skew:                          -0.092   Prob(JB):                     3.89e-07
Kurtosis:                       3.035   Cond. No.                         13.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

При сравнении выборок до теста и после вышли результаты, которые плохо согласуются друг с другом. 1.353 против 0.274

### 3. Построение uplift модели по результатам AB теста
Вам даны результаты AB теста для модели конверсии клиента в продукте (conversion). Конверсия может принимать значения 1 или 0, что значит - клиент подключил услугу, либо не подключил соответственно.

Задача:

1. Оцените ATE по данным.
2. Оцените CATE для каждого наблюдения.
3. Проставьте рекомендации к воздействию на каждый объект.
4. Какая ожидаемая конверсия при оптимальном воздействии?
5. Все выводы, предпосылки и методы представляйте в явном виде в ноутбуке.

Визуализация результатов также будет приветствоваться.

dataset_part3_students.csv

In [ ]:
data = pd.read_csv('{}/dataset_part3_students.csv'.format(DATA_PATH), sep='\t')
data.head()

,treatment,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,conversion
0,control,0.034351,0.550725,-1.348524,-0.029591,0.499535,-0.412464,0.933588,-0.522954,-0.029591,-0.807218,1.318646,-0.065262,-0.956947,-0.314573,-0.879456,0.640264,1.165722,0.570495,0.529443,0.227962,1.929914,0.798664,0
1,control,-1.059404,0.299404,0.220273,-1.569660,1.084204,-2.167118,1.216469,-0.744547,-1.569660,-0.460971,-1.396720,0.600044,1.000000,0.303664,1.080643,2.646483,-0.329690,1.725636,1.434649,-2.064893,0.188000,-1.134651,0
2,control,0.051934,2.338729,-0.341017,-1.099362,0.693090,-2.424267,1.572066,-0.212798,-1.099362,-1.430502,0.469694,1.251940,0.943003,-0.083721,-1.981712,0.224777,2.038004,2.516398,2.324280,-3.182196,-0.888490,-1.402561,0
3,treatment,0.667770,0.539316,0.468719,-0.840885,-0.708331,0.126634,1.275854,-0.954919,-0.840885,0.408691,-0.422736,-1.593407,0.247038,-0.752011,-0.347319,-1.659402,-0.871980,-0.893513,-0.658078,-2.704997,-0.390745,-1.578004,1
4,control,-1.416158,2.486628,0.496402,-1.174618,2.738267,-4.756755,-0.180918,-1.281432,-1.174618,-0.243256,0.598662,0.750706,0.554053,-1.198151,-0.232961,0.948875,1.210523,1.536723,1.548610,-0.926435,0.972459,-0.711251,0


In [ ]:
y = data['conversion']

D = np.zeros((data['treatment'].shape[0],))

for i in range(data['treatment'].shape[0]):
  D[i] = 0.0 if data['treatment'][i] == 'control' else 1.0

#D = data['treatment'].values
X = data.iloc[:, 1:-1]

In [ ]:
lr = LRSRegressor()
te, lb, ub = lr.estimate_ate(X, D, y)
print('Average Treatment Effect (Linear Regression): {:.2f} ({:.2f}, {:.2f})'.format(te[0], lb[0], ub[0]))

Average Treatment Effect (Linear Regression): 0.06 (0.06, 0.06)


In [ ]:
causal = CausalModel(
    Y=y, D=D, X=X
)

causal.est_via_ols(adj=1)
print(causal.estimates)

/usr/local/lib/python3.6/dist-packages/causalinference/estimators/ols.py:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  olscoef = np.linalg.lstsq(Z, Y)[0]



Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      0.060      0.001     45.224      0.000      0.058      0.063



Оценка ATE дала 0.06, что говорит о положительном влиянии